# Object Detection with Unbihexium

[![CI](https://github.com/unbihexium-oss/unbihexium/workflows/CI/badge.svg)](https://github.com/unbihexium-oss/unbihexium/actions)
[![PyPI](https://img.shields.io/pypi/v/unbihexium.svg)](https://pypi.org/project/unbihexium/)
[![License](https://img.shields.io/badge/license-Apache--2.0-blue.svg)](LICENSE.txt)

**Author**: Unbihexium OSS Foundation  
**Version**: 1.0.0

---

## Purpose

This notebook demonstrates object detection using Unbihexium models for:
- Ship detection
- Building detection
- Aircraft detection
- Vehicle detection

## Mathematical Background

Object detection uses bounding box regression and classification:

$$\text{IoU}(A, B) = \frac{|A \cap B|}{|A \cup B|}$$

$$\text{mAP} = \frac{1}{|C|} \sum_{c \in C} \text{AP}(c)$$

In [ ]:
import numpy as np
from pathlib import Path

try:
    import onnxruntime as ort
    print(f"ONNX Runtime version: {ort.__version__}")
except ImportError:
    print("Install: pip install onnxruntime")

## Available Detection Models

| Model | Task | Classes | Variants |
|-------|------|---------|----------|
| ship_detector | Maritime | 1 | tiny, base, large |
| building_detector | Urban | 1 | tiny, base, large |
| aircraft_detector | Aerial | 1 | tiny, base, large |
| vehicle_detector | Ground | 1 | tiny, base, large |

In [ ]:
# Load ship detector
model_path = Path("../model_zoo/assets/tiny/ship_detector_tiny/model.onnx")

if model_path.exists():
    session = ort.InferenceSession(str(model_path))
    print(f"Loaded: {model_path.name}")
    
    # Create synthetic input
    input_shape = session.get_inputs()[0].shape
    print(f"Expected input shape: {input_shape}")
    
    # Simulate satellite image tile
    np.random.seed(42)
    dummy_image = np.random.rand(1, 3, 32, 32).astype(np.float32)
    
    # Run detection
    outputs = session.run(None, {"input": dummy_image})
    print(f"Detection output shape: {outputs[0].shape}")
else:
    print("Model not found. Run from repository root.")

## Post-Processing: Non-Maximum Suppression

Apply NMS to filter overlapping detections:

In [ ]:
def compute_iou(box1, box2):
    """Compute Intersection over Union."""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    inter = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - inter
    
    return inter / union if union > 0 else 0

# Example boxes
box_a = [10, 10, 50, 50]
box_b = [20, 20, 60, 60]
print(f"IoU between boxes: {compute_iou(box_a, box_b):.3f}")

## Next Steps

- [Spectral Indices](03_spectral_indices.ipynb)
- [Change Detection](04_change_detection.ipynb)

---

**Copyright 2025 Unbihexium OSS Foundation. Apache-2.0 License.**